In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/stock_price_4_fun/notebook


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)
import stumpy
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import numpy.testing as npt
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import os
from vnstock import * #import all functions, including functions that provide OHLC data for charting
# from vnstock.chart import * # import chart functions

In [3]:
df = pd.read_csv('data/BaoCaoLaiLo_058C647873.csv', encoding='latin1', skiprows=5)

# Step 3: Rename columns according to the specified order
column_order = ['date', 'sell_vol', 'sell_price', 'sell_value', 'capital_price', 'capital_value', 'dividend', 'win_loss_value', 'win_loss_percent']
df.columns = column_order
df = df.drop(df.index[-1])

# Step 4: Display the resulting dataframe
print(df.head())

         date sell_vol sell_price  sell_value capital_price capital_value  \
0         BCG    1,200      9,780  11,736,000        10,150    12,180,000   
1  23/06/2023      600      9,950   5,970,000        10,150     6,090,000   
2  26/06/2023      600      9,610   5,766,000        10,150     6,090,000   
3         BMP      460     77,333  35,573,400        79,068    36,371,499   
4  30/05/2023        5     78,800     394,000        78,390       391,952   

  dividend win_loss_value win_loss_percent  
0        0       -444,000           -3.65%  
1      NaN       -120,000           -1.97%  
2      NaN       -324,000           -5.32%  
3        0       -798,099           -2.19%  
4      NaN          2,048            0.52%  


In [4]:
df.head()

,date,sell_vol,sell_price,sell_value,capital_price,capital_value,dividend,win_loss_value,win_loss_percent
0,BCG,"1,200","9,780","11,736,000","10,150","12,180,000",0,"-444,000",-3.65%
1,23/06/2023,600,"9,950","5,970,000","10,150","6,090,000",NaN,"-120,000",-1.97%
2,26/06/2023,600,"9,610","5,766,000","10,150","6,090,000",NaN,"-324,000",-5.32%
3,BMP,460,"77,333","35,573,400","79,068","36,371,499",0,"-798,099",-2.19%
4,30/05/2023,5,"78,800","394,000","78,390","391,952",NaN,"2,048",0.52%


In [5]:
def preprocess_percent_sign(df, col ='win_loss_percent'):
    df[col] = df[col].str.rstrip('%')
    df[col] = df[col].astype('float32')
    return df

In [6]:
# Step 4: Remove the '%' symbol from the 'win_loss_percent' column
df = preprocess_percent_sign(df)

In [7]:
df.head()

,date,sell_vol,sell_price,sell_value,capital_price,capital_value,dividend,win_loss_value,win_loss_percent
0,BCG,"1,200","9,780","11,736,000","10,150","12,180,000",0,"-444,000",-3.65
1,23/06/2023,600,"9,950","5,970,000","10,150","6,090,000",NaN,"-120,000",-1.97
2,26/06/2023,600,"9,610","5,766,000","10,150","6,090,000",NaN,"-324,000",-5.32
3,BMP,460,"77,333","35,573,400","79,068","36,371,499",0,"-798,099",-2.19
4,30/05/2023,5,"78,800","394,000","78,390","391,952",NaN,"2,048",0.52


In [ ]:
def create_stock_nane_from_date(df):
    df['stock_name'] = df['date'].where(df['date'].str.len() == 3)
    df['stock_name'] = df['stock_name'].ffill().astype('category')
    df = df[df['date'].str.len() != 3]
    return df

In [11]:
def convert_type(df):
    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
    df = df.drop('dividend', axis=1)
    return df

In [12]:
df = convert_type(df)

In [17]:
df.columns

Index(['date', 'sell_vol', 'sell_price', 'sell_value', 'capital_price',
       'capital_value', 'win_loss_value', 'win_loss_percent', 'stock_name'],
      dtype='object')

In [19]:
num_cols = ['sell_vol', 'sell_price', 'sell_value', 'capital_price',
       'capital_value', 'win_loss_value', 'win_loss_percent']
df[num_cols] = df[num_cols].replace(',', '', regex=True)

# Convert columns to numeric
df[num_cols] = df[num_cols].apply(pd.to_numeric)
for col in num_cols:
    df[col] = df[col].astype(float)

In [20]:
df.head(20)

,date,sell_vol,sell_price,sell_value,capital_price,capital_value,win_loss_value,win_loss_percent,stock_name
1,2023-06-23,600.0,9950.0,5970000.0,10150.0,6090000.0,-120000.0,-1.97,BCG
2,2023-06-26,600.0,9610.0,5766000.0,10150.0,6090000.0,-324000.0,-5.32,BCG
4,2023-05-30,5.0,78800.0,394000.0,78390.0,391952.0,2048.0,0.52,BMP
5,2023-05-31,100.0,80000.0,8000000.0,78390.0,7839038.0,160962.0,2.05,BMP
6,2023-06-02,100.0,77000.0,7700000.0,79269.0,7926904.0,-226904.0,-2.86,BMP
7,2023-06-02,5.0,77800.0,389000.0,79269.0,396345.0,-7345.0,-1.85,BMP
8,2023-06-02,1.0,77800.0,77800.0,79269.0,79269.0,-1469.0,-1.85,BMP
9,2023-06-02,49.0,77400.0,3792600.0,79269.0,3884183.0,-91583.0,-2.36,BMP
10,2023-06-02,200.0,76100.0,15220000.0,79269.0,15853808.0,-633808.0,-4.00,BMP
12,2023-06-08,400.0,17400.0,6960000.0,17400.0,6960000.0,0.0,0.00,BSR


In [21]:
def my_strategy_analyse(df):
    """
    Analyze trading strategy performance metrics.

    Parameters:
    - df (pd.DataFrame): DataFrame containing trading data.

    Returns:
    - dict: Dictionary containing calculated metrics.
    """

    # Calculate win ratio
    total_trades = len(df)
    winning_trades = len(df[df['win_loss_value'] > 0])
    win_ratio = winning_trades / total_trades

    # Calculate payoff ratio
    average_win = df[df['win_loss_value'] > 0]['win_loss_value'].mean()
    average_loss = df[df['win_loss_value'] < 0]['win_loss_value'].mean()
    payoff_ratio = abs(average_win / average_loss)

    # Calculate largest winning trade
    largest_winning_trade = df['win_loss_value'].max()

    # Calculate largest losing trade
    largest_losing_trade = df['win_loss_value'].min()

    # Calculate average winning trade
    average_winning_trade = df[df['win_loss_value'] > 0]['win_loss_value'].mean()

    # Calculate average losing trade
    average_losing_trade = df[df['win_loss_value'] < 0]['win_loss_value'].mean()

    # Calculate largest % drawdown
    cumulative_returns = (1 + df['win_loss_percent'] / 100).cumprod()
    largest_drawdown = (cumulative_returns / cumulative_returns.cummax() - 1).min()

    # Calculate average % drawdown
    average_drawdown = (cumulative_returns / cumulative_returns.cummax() - 1).mean()

    # Calculate annualized profit/loss
    total_days = (df['date'].max() - df['date'].min()).days
    annualized_return = (cumulative_returns.iloc[-1]) ** (365 / total_days) - 1

    # Create a dictionary to store the calculated metrics
    metrics = {
        'Win Ratio': win_ratio,
        'Payoff Ratio': payoff_ratio,
        'Largest Winning Trade': largest_winning_trade,
        'Largest Losing Trade': largest_losing_trade,
        'Average Winning Trade': average_winning_trade,
        'Average Losing Trade': average_losing_trade,
        'Largest % Drawdown': largest_drawdown * 100,  # Convert to percentage
        'Average % Drawdown': average_drawdown * 100,  # Convert to percentage
        'Annualized Profit/Loss': annualized_return * 100  # Convert to percentage
    }

    return metrics

In [22]:
metrics = my_strategy_analyse(df)

In [23]:
metrics

{'Win Ratio': 0.6145038167938931,
 'Payoff Ratio': 1.3901079930969387,
 'Largest Winning Trade': 2057498.0,
 'Largest Losing Trade': -1906250.0,
 'Average Winning Trade': 330680.5714285714,
 'Average Losing Trade': -237881.21,
 'Largest % Drawdown': -57.65596273317476,
 'Average % Drawdown': -17.982174851701384,
 'Annualized Profit/Loss': 175546.97664066326}

In [25]:
group_df = df.groupby('stock_name')
group_df.head()

,date,sell_vol,sell_price,sell_value,capital_price,capital_value,win_loss_value,win_loss_percent,stock_name
1,2023-06-23,600.0,9950.0,5970000.0,10150.0,6090000.0,-120000.0,-1.97,BCG
2,2023-06-26,600.0,9610.0,5766000.0,10150.0,6090000.0,-324000.0,-5.32,BCG
4,2023-05-30,5.0,78800.0,394000.0,78390.0,391952.0,2048.0,0.52,BMP
5,2023-05-31,100.0,80000.0,8000000.0,78390.0,7839038.0,160962.0,2.05,BMP
6,2023-06-02,100.0,77000.0,7700000.0,79269.0,7926904.0,-226904.0,-2.86,BMP
...,...,...,...,...,...,...,...,...,...
293,2023-05-24,800.0,9660.0,7728000.0,9667.0,7733677.0,-5677.0,-0.07,VTO
294,2023-05-24,200.0,9660.0,1932000.0,9667.0,1933419.0,-1419.0,-0.07,VTO
295,2023-05-29,500.0,9680.0,4840000.0,9665.0,4832251.0,7749.0,0.16,VTO
296,2023-05-29,700.0,9660.0,6762000.0,9665.0,6765151.0,-3151.0,-0.05,VTO
